In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_context('notebook', font_scale=1.5)

In [2]:
import warnings
warnings.simplefilter('ignore', FutureWarning)

**1**. (25 points)

In this exercise, we will practice using Pandas dataframes to explore and summarize a data set `heart`.

This data contains the survival time after receiving a heart transplant, the age of the patient and whether or not the survival time was censored

- Number of Observations - 69
- Number of Variables - 3

Variable name definitions::

- survival - Days after surgery until death
- censors - indicates if an observation is censored. 1 is uncensored
- age - age at the time of surgery

Answer the following questions (5 points each) with respect to the `heart` data set:

- How many patients were censored?
- What is the correlation coefficient between age and survival for uncensored patients? 
- What is the average age for censored and uncensored patients?
- What is the average survival time for censored and uncensored patients under the age of 45?
- What is the survival time of the youngest and oldest uncensored patient?


In [4]:
import statsmodels.api as sm
heart = sm.datasets.heart.load_pandas().data
heart.head(n=6)

,survival,censors,age
0,15.0,1.0,54.3
1,3.0,1.0,40.4
2,624.0,1.0,51.0
3,46.0,1.0,42.5
4,127.0,1.0,48.0
5,64.0,1.0,54.6


In [21]:
# How many patients were censored?
number_censored = heart.groupby('censors').size()[0]
print("{} patients were censored".format(number_censored))


24 patients were censored


In [31]:
#correlation ccoefficient between age and survivial for uncensored patients

#Select uncensored patients
uncensored = heart[heart.censors == 1]
uncensored = uncensored.drop(['censors'],axis=1)
corr = uncensored.age.corr(uncensored.survival)
print("{} is the correlation coefficient between age and survival for uncensored patients".format(corr))

0.003256499283211932 is the correlation coefficient between age and survival for uncensored patients


In [37]:
#What is the average age for censored and uncensored patients?
censor_group = heart.groupby('censors')
ave_age = censor_group.age.mean()
ave_age

censors
0.0    41.729167
1.0    48.484444
Name: age, dtype: float64

In [44]:
#what is the average survival time for censored and uncensored patients under the age of 45?
age_filtered = heart[heart.age<45]
age_filtered.groupby('censors').survival.mean()

censors
0.0    712.818182
1.0    169.909091
Name: survival, dtype: float64

In [67]:
#What is the survival time of the youngest and oldest uncensored patient?
survival_time_oldest = heart[heart.age == uncensored.age.max()].survival.values

survival_time_youngest = heart[heart.age == uncensored.age.min()].survival.values

print("survival time for oldest uncensored patient is as below:")
print(survival_time_oldest)
print("survival time for oldest uncensored patient is as below:")
print(survival_time_youngest)


survival time for oldest uncensored patient is as below:
[60.]
survival time for oldest uncensored patient is as below:
[228.]


**2**. (35 points)

- Consider a sequence of $n$ Bernoulli trials with success probabilty $p$ per trial. A string of consecutive successes is known as a success *run*. Write a function that returns the counts for runs of length $k$ for each $k$ observed in a dictionary. (10 points)

For example: if the trials were [0, 1, 0, 1, 1, 0, 0, 0, 0, 1], the function should return 
```
{1: 2, 2: 1})
```

Test that it does so.

- What is the probability of observing at least one run of length 5 or more when $n=100$ and $p=0.5$?. Estimate this from 100,000 simulated experiments. Is this more, less or equally likely than finding runs of length 7 or more when $p=0.7$? (10 points)

- There is an exact solution

$$
s_n = \sum_{i=1}^n{f_i} \\
f_n = u_n - \sum_{i=1}^{n-1} {f_i u_{n-i}} \\
u_n = p^k - \sum_{i=1}^{k-1} u_{n-i} p_i
$$

Implement the exact solution using caching to avoid re-calculations and calculate the same two probabilities found by simulation. (15 points)

In [408]:
def successrun(trials):
    """This function returns a dictionary which contains counts for runs of lenth k for each k observed in a dictionary"""
    
    finalresult = {}
    count = 0
   
    
    
    for i in trials:
        if i == 1:
            count = count + 1
        else:
            if count != 0:
                finalresult[count] = finalresult.get(count, 0) + 1
                count = 0
    
    if i == 1:
        finalresult[count] = finalresult.get(count, 0) + 1
    
    return finalresult
    
    


    


    


In [409]:
run = [np.random.binomial(1,0.5) for i in range(10)]

print(run)
print(successrun(run))



        

[1, 0, 0, 1, 0, 0, 0, 0, 1, 0]
{1: 3}


In [410]:
#Simulations

def simu(n,p,simulation,k):
    count = 0
    total = 0
    
    for i in range(simulation):
        series = [np.random.binomial(1,p) for i in range(n)]
       
        runcount = successrun(series)
       
        for j in runcount.keys():
            if j>=k:
                total = total + 1
                break
     
    
    return total/simulation
            
                
    

In [411]:
print("probability of having at least one run of length 5 or more for n=100 and p=0.5",simu(100,0.5,100000,5))
print("probability of having at least one run of length 7 or more for n=100 and p=0.7",simu(100,0.7, 100000,7))

probability of having at least one run of length 5 or more for n=100 and p=0.5 0.81189
probability of having at least one run of length 7 or more for n=100 and p=0.7 0.94919


In [429]:
from functools import lru_cache

@lru_cache()
def u(n, k, p):
    """a recursive function for equation of u"""
    #function returns 0 when the number of bernoulli trials is smaller than target run length
    if n < k:
        return 0
    return p**k - sum(u(n-i, k, p) * p**i for i in range(1, k))

@lru_cache()
def f(n, k, p):
    """a recursive function for equation of f """
    return u(n, k, p) - sum(f(i, k, p) * u(n-i, k, p) for i in range(1, n))


@lru_cache() 
def s(n, k, p):
    """a recursive function for equation of s"""
    return sum(f(i, k, p) for i in range(1, n+1))




In [430]:
print("Probability of observing at least one run of length 5 or more for n=100 and p=0.5",s(k=5,n=100,p=.5))
print("Probability of observing at least one run of length 7 or more for n=100 and p=0.7",s(k=7,n=100,p=.7))

Probability of observing at least one run of length 5 or more for n=100 and p=0.5 0.8101095991963579
Probability of observing at least one run of length 7 or more for n=100 and p=0.7 0.9491817984156692


**3**. (40 points)

Given matrix $M$
```python
      [[7, 8, 8],
       [1, 3, 8],
       [9, 2, 1]]
```

- Normalize the given matrix $M$ so that all rows sum to 1.0.  (5 points)
- The normalized matrix can then be considered as a transition matrix $P$ for a Markov chain. Find the stationary distribution of this matrix in the following ways using `numpy` and `numpy.linalg` (or `scipy.linalg`):
    - By repeated matrix multiplication of a random probability vector $v$ (a row vector normalized to sum to 1.0) with $P$ using matrix multiplication with `np.dot`. (5 points)
    - By raising the matrix $P$ to some large power until it doesn't change with higher powers (see `np.linalg.matrix_power`) and then calculating $vP$ (10 points)
    - From the equation for stationarity $wP = w$, we can see that $w$ must be a left eigenvector of $P$ with eigenvalue $1$ (Note: np.linalg.eig returns the right eigenvectors, but the left eigenvector of a matrix is the right eigenvector of the transposed matrix). Use this to find $w$ using `np.linalg.eig`. (20 points)

Suppose $w = (w_1, w_2, w_3)$. Then from $wP = w$, we have:
\begin{align}
w_1 P_{11} + w_2 P_{21} + w_3 P_{31} &= w_1 \\
w_1 P_{12} + w_2 P_{22} + w_3 P_{32} &= w_2 \\
w_1 P_{13} + w_2 P_{23} + w_3 P_{331} &= w_3 \\
\end{align}
This is a singular system, but we also know that $w_1 + w_2 + w_3 = 1$. Use these facts to set up a linear system of equations that can be solved with `np.linalg.solve` to find $w$.

In [274]:
#normalize Matrix M
matrix = np.array([[7, 8, 8],
       [1, 3, 8],
       [9, 2, 1]])

total = matrix.sum(axis=1).reshape(-1,1)
matrix = matrix/total
matrix






array([[0.30434783, 0.34782609, 0.34782609],
       [0.08333333, 0.25      , 0.66666667],
       [0.75      , 0.16666667, 0.08333333]])

In [275]:
#part 1
v = np.random.random([1,3])
totalv = v.sum()
v = v/totalv
print("original v is: \n {}".format(v))

vold = v
vnew = np.dot(vold, matrix)
roundno = 0

while (vold == vnew).all() == False:
    vold = vnew
    vnew = np.dot(vold,matrix)
    roundno = roundno + 1

    
print("stationary v is: \n {}".format(vnew))


    
    





original v is: 
 [[0.08739819 0.14128984 0.77131197]]
stationary v is: 
 [[0.39862184 0.2605972  0.34078096]]


In [276]:
#part 2
tol = 1e-06
matrixold = matrix
matrixnew = np.linalg.matrix_power(matrix,2)
power = 2

compare = False

while (matrixold == matrixnew).all() == False:
    matrixold = matrixnew
    power = power + 1
    matrixnew = np.linalg.matrix_power(matrix, power)
  

prod = np.dot(vnew, matrixnew)
print("stationary matrix P is: \n {}".format(matrixnew))
print("Product vP is: \n {}".format(prod))




stationary matrix P is: 
 [[0.39862184 0.2605972  0.34078096]
 [0.39862184 0.2605972  0.34078096]
 [0.39862184 0.2605972  0.34078096]]
Product vP is: 
 [[0.39862184 0.2605972  0.34078096]]


In [301]:
#part 3
eigenvals,eigenvecs = np.linalg.eig(matrix.T)
index = np.where(eigenvals == 1)
w = np.real(eigenvecs[:, index]).T.reshape(3,)

w = w/w.sum()
print("w is {}".format(w))


w is [0.39862184 0.2605972  0.34078096]


In [303]:
#part 4
C = P.T - np.eye(3)
C[2] = [1,1,1]
result = np.linalg.solve(C, [0,0,1])
print("stationary vector: \n {}".format(result))

stationary vector: 
 [0.39862184 0.2605972  0.34078096]
